In [1]:
# THIS IS ALL SETUP STUFF

# Import ecessary libraries
import tweepy
import requests
import json
import os
import praw
import time

# Get authentication data from local JSON file (keep secret for presenting)
with open('keys.json') as f:
    api_keys = json.loads(f.read())
access_token = api_keys['access_token']
access_token_secret = api_keys['access_token_secret']
consumer_key = api_keys['consumer_key']
consumer_key_secret = api_keys['consumer_key_secret']

# Initialize Twitter API with appropriate auth keys/tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [2]:
def get_image_url_and_name():
    """
    Using PRAW, this function will access all posts from /r/memes and return a list of 
    the title, url, and author of the latest "hot" thread with a direct image url 
    (i.e. 'redd.it/...')
    """
    with open('reddit_api_keys.json') as f:
        reddit_api_keys = json.loads(f.read())
    reddit = praw.Reddit(client_id=reddit_api_keys['client_id'],
                        client_secret=reddit_api_keys['client_secret'],
                        password = reddit_api_keys['password'],
                        user_agent = 'Gary and Jayam\'s twitter bot project @ UIUC',
                        username=reddit_api_keys['username'])
    memes = reddit.subreddit('memes')
    for submission in memes.hot():
        if 'redd' in submission.url:
            return([submission.title, submission.url, submission.author])

In [3]:
# Given a direct image URL (www.i.<some_image_host>.com) downloads photo
def download_image(img_url):
    request = requests.get(img_url, stream=True)
    if request.status_code == 200:    # Status code 200 = download successful
        filename = 'temp.jpg'         # Use temp.jpg to save image, overrides each time this function is called
        with open(filename, 'wb') as image:
            for chunk in request:
                image.write(chunk)
        return filename
    else:                            # Error handling in case something messes up
        print("Download error. Status code: %d" % request.status_code)
        return None

In [4]:
# Given the image URL, downloads image and posts to twitter
def post_to_twitter(img_url, caption):
    img_file = download_image(img_url)
    if(img_file is not None):
        api.update_with_media(img_file, status=caption)
    else:
        print("Something went wrong, could not post to Twitter.")

In [ ]:
while True:
    reddit_info = get_image_url_and_name()
    post url = reddit_info[1]
    message = reddit_info[0] + " - posted by /u/" + str(reddit_info[2])
    post_to_twitter(reddit_info[1], reddit_info[0])
    time.sleep(3 * 3600) # Take a 3 hour nap then do it again